# CH8. Blog 앱 확장 - 검색 기능

: Q-객체를 이용하여 테이블의 복잡한 쿼리 처리하기

* 검색기능을 위한 검색 단어를 입력받는 폼 기능 및 Q-객체를 이용한 검색 단어가 들어있는 블로그 찾기 및 출력하기

> **화면 UI 설계**   
> Search 메뉴 : 검색 폼 보여주기   
> Submit 메뉴 : 검색 결과 출력   
> 검색 폼과 검색 결과를 같은 페이지에서 구현   

> **테이블 설계**   
> 별도의 정의 없음

> **URL 설계**   
> /blog/serach/ - SearchFormView(FormView) - post_search.html

> **작업 순서**   
> urls.py : URL 정의 추가   
> views.py : 로직 정의 추가   
> 템플릿 디렉터리 : 템플릿 수정 및 추가   
> forms.py : 검색 폼 클래스 정의

## 1. URL 코딩

: blog/search/ 경로 추가

```
blog/urls.py

from django.urls import path, re_path
from blog import views

app_name = 'blog'

urlpatterns = [
    # 2가지 요청을 처리하는 PostLV
    path('', views.PostLV.as_view(), name = 'index'),
    path('post/', views.PostLV.as_view(), name = 'post_list'),
    re_path(r'^post/(?P<slug>[-\w]+)/$', views.PostDV.as_view(), name = 'post_detail'),
    path('archive/', views.PostAV.as_view(), name = 'post_archive'),
    path('archive/<int:year>/', views.PostYAV.as_view(), name = 'post_year_archive'),
    path('archive/<int:year>/<str:month>/', views.PostMAV.as_view(), name = 'post_month_archive'),
    path('archive/<int:year>/<str:month>/<int:day>/', views.PostDAV.as_view(), name = 'post_day_archive'),
    path('archive/today/', views.PostTAV.as_view(), name = 'post_today_archive'),
    path('tag/', views.TagCloudTV.as_view(), name = 'tag_cloud'),
    path('tag/<str:tag>/', views.TaggedObjectLV.as_view(), name = 'tagged_object_list'),
    path('serach/', views.SearchFormView.as_view(), name = 'search'),
]

```

## 2. 뷰 코딩하기

: forms.py에서 폼을 정의한 후, urls.py에서 정의한 SearchFormView 추가하기

```
blog/forms.py

# 뷰를 정의하는 것처럼 클래스 외부 정의
# 모델을 정의하는 것처럼 클래스 내부 정의

from django import forms

class PostSearchForm(forms.Form) :
    search_word = forms.CharField(label = 'SearchWord')
```

```
blog/views.py

class SearchFormView(FormView) :
    form_class = PostSearchForm
    template_name = 'blog/post_search.html'
    
    def form_vaild(self, form) :
        searchWord = form.cleaned_data['search_word']
        post_list = Post.objects.filter(Q(title__icontains = searchWord) | Q(description__icontains = serachWord) | Q(content__icontains = searchWord)).distinct()
        
        context = {}
        context['form'] = form
        context['search_terms'] = searchWord
        context['object_list'] = post_list
        
        return render(self.request, slef.template_name, context)
```

## 3. 템플릿 코딩하기

**1. base.html 수정**

: Search 메뉴에 링크 연결하기

```
templates/base.html

# 드랍다운 네비게이션 바 수정

<li class="nav-item dropdown mx-1 btn btn-link">
              <a class = "nav-link dropdown-toggle text-white" href = "#" data-toggle = "dropdown"> Others </a>
              <div class = "dropdown-menu">
                <a class = "dropdown-item" href = "{% url 'admin:index' %}"> Admin </a>
                <div class = "dropdown-divider"> </div>
                <a class = "dropdown-item" href = "{% url 'blog:post_archive' %}"> Archive </a>
                <a class = "dropdown-item" href = "{% url 'blog:search' %}"> Search </a>
              </div>
            </li>
```

**2. post_search.html**

: 검색 폼과 검색 결과를 한 화면에 보여주기

```
blog/templates/blog/post_search.html

<!DOCTYPE html>
<html>

<body>

    {% extends "base.html" %}

    {% block title %} Searching Post {% endblock %}

    {% block content %}
    
        <h1> Blog Search </h1>
        <br>

        <form action = "." method = "post">
            {% csrf_token %}
            {{ form.as_table }}
            <input type = "submit" value = "Submit" class = "btn btn-primary btn-sm">
        </form>

        <br/><br/>

        {% if object_list %}
            {% for post in object_list %}
                <h2><a href = '{{ post.get_absolute_url }}'> {{ post.title }} </a></h2>
                {{ post.modify_date | date:"N d, Y" }}
                <p> {{ post.description }} </p>
            {% endfor %}
        {% elif search_term %}
            <b><i> Search Word({{ search_term }}) Not Found! </i></b>
        {% endif %}

    {%end block %}
</body>
</html>
```

## 확인하기

http://localhost:8000